In [1]:
%load_ext autoreload
%autoreload 2
from hloc.my_localize_lib import reconstruction_from_dataset, localize_image
from hloc.utils import viz_3d


/home/unicorn/project/Hierarchical-Localization/hloc/my_localize_lib.py:17: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# 重建
model = reconstruction_from_dataset('datasets/My_lib_dataset/mapping4/', 4)

[2023/02/23 08:56:43 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2023/02/23 08:56:43 hloc INFO] Found 9 images in root datasets/My_lib_dataset/mapping4.
[2023/02/23 08:56:43 hloc INFO] Skipping the extraction.
[2023/02/23 08:56:43 hloc INFO] Extracting image pairs from a retrieval database.
[2023/02/23 08:56:48 hloc INFO] Found 40 pairs.
[2023/02/23 08:56:48 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
[2023/02/23 08:56:48 hloc INFO] Found 9 images in root datasets/My_lib_dataset/mapping4.
[2023/02/23 08:56:48 hloc INFO] Skipping the extraction.
[2023/02/23 08:56:48 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations'

reload matcher model
Loaded SuperGlue model ("outdoor" weights)


100%|██████████| 4/4 [00:02<00:00,  1.43it/s]
[2023/02/23 08:56:51 hloc INFO] Finished exporting matches.
[2023/02/23 08:56:51 hloc WARNING] The database already exists, deleting it.
[2023/02/23 08:56:51 hloc INFO] Creating an empty database...
[2023/02/23 08:56:51 hloc INFO] Importing images into the database...
[2023/02/23 08:56:51 hloc INFO] Importing features into the database...
100%|██████████| 9/9 [00:00<00:00, 1246.04it/s]
[2023/02/23 08:56:51 hloc INFO] Importing matches into the database...
 70%|███████   | 28/40 [00:00<00:00, 1681.25it/s]


KeyError: 'tmp.jpg'

In [3]:
model # superpoint_inloc

NameError: name 'model' is not defined

In [4]:
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping")
fig.show()

In [ ]:
for item in model.images:
    print(model.images[item].name)

In [3]:
%load_ext autoreload
%autoreload 2
from hloc.my_localize_lib import reconstruction_from_dataset, localize_image
from hloc.utils import viz_3d
import pycolmap
from hloc import visualization
from pathlib import Path
import time

/home/unicorn/project/Hierarchical-Localization/hloc/my_localize_lib.py:17: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [4]:
dataset = Path('datasets/My_lib_dataset/mapping3')
start = time.time()
camera = pycolmap.infer_camera_from_image(dataset/'../query/room10.jpg')
ret, model, log, _ = localize_image(dataset, dataset/'../query', image_query_name='room10.jpg')
end = time.time()
print('Used time: ', end-start, 's')

if ret['success'] is True:
    print("Localization success! ")

    # fig = viz_3d.init_figure()
    # viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping")
    # pose = pycolmap.Image(tvec=ret['tvec'], qvec=ret['qvec'])
    # viz_3d.plot_camera_colmap(fig, pose, camera, color='rgba(0,255,0,0.5)', name='query')
    # fig.show()
    # visualization.visualize_loc_from_log(Path(dataset), Path('../query/room2.jpg'), log, model, top_k_db = 3)
else:
    print("Localization failed..")

MainWork Loading...
Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)


[2023/02/24 17:06:45 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}


MainWork Load Success!
configurate for  0.0002758502960205078 s


[2023/02/24 17:06:45 hloc INFO] Skipping the extraction.
[2023/02/24 17:06:45 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
[2023/02/24 17:06:45 hloc INFO] Skipping the extraction.
[2023/02/24 17:06:45 hloc INFO] Extracting image pairs from a retrieval database.
[2023/02/24 17:06:45 hloc INFO] Found 3 pairs.
[2023/02/24 17:06:45 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue', 'sinkhorn_iterations': 5, 'weights': 'outdoor'},
 'output': 'matches-superglue-it5'}
[2023/02/24 17:06:45 hloc INFO] Skipping the matching.


total:  0.7883744239807129 s
Used time:  8.992131233215332 s
Localization success! 


In [ ]:
import sys
print(ret.keys())
print(log.keys())
# for item in ret.keys():
#     print(ret[item].type())
# log
# print(log)
print(log)


In [5]:
ret['qvec'] = ret['qvec'].tolist()
ret['tvec'] = ret['tvec'].tolist()
ret['camera']['params'] = ret['camera']['params'].tolist()
log['keypoints_query'] = log['keypoints_query'].tolist()

In [6]:
import numpy as np
import json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [8]:
import json
with open('json.txt', 'w') as fp:
    json.dump(log, fp, cls=NpEncoder)

In [17]:
with open('log.txt', 'w') as fp:
    fp.write(log.)

{'db': [32, 30, 15], 'PnP_ret': {'success': True, 'qvec': [0.5156002098169827, 0.11491464006066805, 0.7790593665609333, 0.33766485234635796], 'tvec': [-2.829553900717622, -1.0736065858445707, 2.2367822109091375], 'num_inliers': 397, 'inliers': [False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, 

In [ ]:
visualization.visualize_sfm_2d(model, Path('datasets/My_lib_dataset/mapping/'), color_by='visibility', selected=[80])

In [4]:
import pycolmap 
from hloc.utils import viz_3d

In [5]:
reference_sfm = pycolmap.Reconstruction('datasets/My_lib_dataset/mapping3_model/')

In [6]:
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, reference_sfm, color='rgba(255,0,0,0.5)', name="mapping")
fig.show()

In [ ]:
fig.update_traces()